<a href="https://colab.research.google.com/github/shawn-kg/DataMining/blob/main/assign9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Preprocessing

In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
np.random.seed(42)

data = pd.read_csv('./codon_usage.csv')
data = data.drop(data.columns[[1,2,3,4]], axis=1)
data = data.to_numpy()
data = np.delete(data,[486,5063],0)
target = data[:,0]
data = np.delete(data,0,1)
data = data.astype(float)
sss = StratifiedShuffleSplit(n_splits=1,train_size=0.2, random_state=42)
D = 0
T = 0
for ind, _ in sss.split(data,target):
  ind = np.sort(ind)
  D = data[ind]
  T = target[ind]
scaler = MinMaxScaler(copy=False)
scaler.fit_transform(D)

print(D)
print(T)

a=_, b = np.unique(T, return_index = True)
target_dict = dict.fromkeys(T[np.sort(b)], 0)
for i in T:
  target_dict[i]+=1
true_sizes = []

for i in target_dict.keys():
  true_sizes.append(target_dict[i])
target_ind = [0,0,0,0,0,0,0,0,0,0,0]

j=0
for i in target_dict.keys():
  if j!=0:
    target_ind[j]+=target_dict[i] + target_ind[j-1]
  else:
    target_ind[j] = target_dict[i]
  j+=1


def HT(target_dict, D):
  ht = 0
  for i in target_dict.keys():
    ht += (target_dict[i]/D.shape[0])*np.log(target_dict[i]/D.shape[0])
  ht = -ht
  return ht

def pij(k_pred, k, target_ind, cluster_index, n):
  pijs = np.zeros((k_pred,k))
  for c in range(k_pred):
    beg = 0
    end = 0
    for d in range(k):
      count = 0
      if d!=0:
        beg = target_ind[d-1]
      else:
        beg = 0
      end = target_ind[d]
      for ind in range(len(cluster_index[c])):
        if (cluster_index[c][ind] >= beg and cluster_index[c][ind] <end):
          count+=1
      pijs[c][d] = count/n
  hct = 0
  for i in range(k_pred):
    for j in range(k):
      hct += pijs[i][j] * np.log(pijs[i][j] + (.0000000000001))
  hct = -hct
  return hct


def HC(final_clusters, k, n):
  hc = 0
  for h in range(k):
    p = len(final_clusters[h])/n
    if p!=0:
      hc += p * np.log(p)
      # if p==1:
      #   hc += -1
  hc = -hc
  return hc

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[[0.13106819 0.18467134 0.05434706 ... 0.         0.03486529 0.01493388]
 [0.10635327 0.36072198 0.08601998 ... 0.02367257 0.02852615 0.00809394]
 [0.14558995 0.16150323 0.26769645 ... 0.04424779 0.06259905 0.00569995]
 ...
 [0.08163835 0.32179418 0.05349678 ... 0.01902655 0.0570523  0.02040584]
 [0.06623226 0.44733297 0.11769291 ... 0.07876106 0.09429477 0.22993616]
 [0.08275541 0.50161638 0.12272373 ... 0.03451327 0.09033281 0.24635203]]
['vrl' 'vrl' 'vrl' ... 'pri' 'pri' 'pri']


##DENCLUE Algorithm

In [18]:
from pandas.core.resample import h
from pandas.core.arrays.numeric import numbers
import math

def gaussian(x,D,h,d):
  a = x - D
  b = -(np.linalg.norm(a,axis=1)**2)/(2*(h**2))
  t2 = np.exp(b)
  gauss = t2/((math.sqrt(2*math.pi))**d)
  return gauss

def findattractor(x, D, h, e, numPoints, numDims):
  while(True):
    x_prev = np.copy(x)

    g = gaussian(x,D,h,numDims)
    g = g.reshape((-1,1))
    denom = np.sum(g)
    num = np.sum(np.multiply(g,D),axis=0)

    x = num/denom #(mean shift)

    if (np.linalg.norm(x-x_prev)<=e):
      break
  return x

def densityEstimation(x, D, h, numPoints, numDims):
  g = gaussian(x,D,h,numDims)
  fx = np.sum(g)/(numPoints*(h**numDims))
  return fx

def denclue(D,h,ee,e):
  numPoints = D.shape[0]
  numDims = D.shape[1]
  Attractors=[]
  count=0
  for i in range(numPoints):
    xStar = findattractor(D[i],D,h,e, numPoints, numDims)
    if densityEstimation(xStar, D, h, numPoints, numDims)>=ee:
      Attractors.append(xStar)
      count+=1
  C = list(range(D.shape[0]))
  for att in range(len(Attractors)):
    for att2 in range(att+1, len(Attractors)):
      if C[att] == C[att2]:
        continue
      if (np.linalg.norm(Attractors[att]-Attractors[att2])**2 < .0002):
        C[att2] = C[att]

  R = dict()
  for clust in range(len(C)):
    if C[clust] not in R.keys():
      R[C[clust]] = [clust]
    else:
      R[C[clust]].append(clust)

  clusters = []
  print("Number of Clusters: ")
  print(len(R))
  print("Size of Each Cluster")
  for i in R.keys():
    print(len(R[i]))
    clusters.append(R[i])
  return clusters


clusters = denclue(D,0.5,10**-7,0.1)

# NMI
hc = HC(clusters,len(clusters),D.shape[0])

ht = HT(target_dict, D)

hct = pij(len(clusters),11,target_ind,clusters, D.shape[0])

ict = hc + ht - hct

nmi = ict/math.sqrt(hc*ht)

print("NMI")
print(nmi)


Number of Clusters: 
10
Size of Each Cluster
107
1
5
20
131
1
1
2333
1
5
NMI
0.19846115830680033


##Markov Clustering

###Algorithm

In [19]:

# print(D)
R = np.zeros((D.shape[0],1))
for i in range(D.shape[0]):
  R[i] = np.square(D[i]).sum()
A = np.exp((2*np.transpose(np.matmul(D,np.transpose(D)))-R-np.transpose(R))/(2*(.5**2)))
# print(A)

# A = np.array([[1, 1, 0, 1, 0, 1, 0],[1, 1, 1, 1, 0, 0, 0],[0, 1, 1, 1, 0, 0, 1],[1, 1, 1, 1, 1, 0, 0],[0, 0, 0, 1, 1, 1, 1],[1, 0, 0, 0, 1, 1, 1],[0, 0, 1, 0, 1, 1, 1]])


def MarkovClustering(A,r,e):
  # calculate Mt
  i=0
  j=0
  An = A.shape[0]
  Mt = np.zeros((An,An))
  for i in range(A.shape[0]):
    Mt[i] = A[i]/np.sum(A[i])
  t=0
  while (True):
    t+=1
    M_prev = np.copy(Mt)
    Mt = np.dot(M_prev,M_prev)
    Mt_new = Mt**r

    for i in range(Mt.shape[0]):
      Mt_new[i] = Mt_new[i]/np.sum(Mt_new[i])
    Mt = Mt_new
    # Mt = np.around(Mt,40)

    if np.linalg.norm(Mt-M_prev)<=e:
      break
  # print(t)
  return Mt


r = 2.8
while (r<2.9):

  print("R: " + str(r))

  M_final = MarkovClustering(A,r,0.001)
  # print(M_final)


  clustered_nodes = set()
  Adj = list()
  for i in range(M_final.shape[0]):
    Adj.append([])
  for i in range(M_final.shape[0]):
    for j in range(M_final.shape[1]):
      if M_final[i][j]>0:
        Adj[j].append(i)
        clustered_nodes.add(i)

  print("Size of clusters: ")
  count=0
  clusters = []
  cluster_sizes = []
  for c in Adj:
    if len(c)>0:
      cluster_sizes.append(len(c))
      clusters.append(c)
      count+=1
  print(cluster_sizes)
  print("Number of Clusters")
  print(count)

  hc = HC(clusters,len(clusters),A.shape[0])


  hct = pij(len(clusters), 11 ,target_ind,clusters, A.shape[0])


  ict = hc + ht - hct

  nmi = ict/math.sqrt(hc*ht)

  print("NMI:")

  print(nmi)
  r+=.1


R: 2.8
Size of clusters: 
[1, 1, 1, 1, 2, 280, 1, 278, 1, 2, 1, 1, 1, 1789, 1, 2, 1, 1, 1, 1, 1, 36, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 464, 5, 1, 1]
Number of Clusters
38
NMI:
0.23694101527386047
